In [1]:
import pandas as pd


In [12]:
file_name = 'data/Travel-Dataset-5000--master/5000TravelQuestionsDataset.xlsx'
col_names = ['questions', 'a', 'b']
dataset = pd.read_excel(file_name, header=None, names=col_names)
print(dataset.head())
print(dataset.info())

                                           questions    a       b
0  What are the special things we (husband and me...  TTD  TTDSIG
1  What are the companies which organize shark fe...  TTD  TTDOTH
2  Is it safe for female traveller to go alone to...  TGU  TGUHEA
3  What are the best places around Cape Town for ...  TTD  TTDSIG
4  What are the best places to stay for a family ...  ACM  ACMOTH
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   questions  5000 non-null   object
 1   a          5000 non-null   object
 2   b          5000 non-null   object
dtypes: object(3)
memory usage: 117.3+ KB
None


In [13]:
dataset['a'].unique()

array(['TTD', 'TGU', 'ACM', 'TRS', 'WTH', 'FOD', 'ENT', 'TGU\n', 'TTD\n',
       '\nENT'], dtype=object)

In [14]:
dataset['b'].unique()

array(['TTDSIG', 'TTDOTH', 'TGUHEA', 'ACMOTH', 'TRSTRN', 'TGUBAN',
       'WTHTMP', 'TGUNEI', 'TTDSPO', 'ACMAPA', 'TRSOTH', 'TRSAIR',
       'TRSROU', 'TGUOTH', 'ACMRES', 'ACMBUN', 'TRSCRS', 'WTHDRC',
       'TRSRNT', 'TGUTOP', 'FODOTH', 'TGUVIS', 'FODAUT', 'TTDTRI',
       'TGUPLN', 'TRSTAX', 'WTHOTH', 'TRSBUS', 'ACMHOT', 'TGULUG',
       'FODBAK', 'TTDSPA', 'FODBRE', 'TGUATT', 'ENTCLB', 'TGUAVE',
       'TGUTEL', 'TGUCIG', 'TTDSHP', 'TRSTCD', 'ACMBEA', 'FODCOT',
       'TGUAPT', 'TRSLIC', 'TGULAU', 'TTDGYM', 'FODCAT', 'FODBAR',
       'TGUHOL', 'ENTFES', 'TGURUL', 'TGURES', 'WTHTMP\n', 'ACMCAR',
       'FODFMA', 'ENTSHW', 'TRSGAS', '\nTGULAU', 'TRSOTH\n', 'WTHSNW',
       'ENTMUS', 'ENTSPO', 'FODBAK\n', 'TRSAIR\n', 'TGUWEB', 'TRSDRV',
       'FODFCA', 'TGUCIG\n', 'ENTOTH', 'TTDOTH\n', 'WTHOTH\n', 'TTDSIG\n',
       'TGUOTH\n', 'TTDSHP\n', 'TRSROU\n', 'TTDSPO\n', '\nACMOTH',
       'ACMOTH\n', '\nWTHOTH'], dtype=object)